<a href="https://colab.research.google.com/github/WinuWijegunawardhana98/Integrated-Healthcare-Management-System-for-Vulnerable-Populations/blob/winodya/Drug_Consumption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pytesseract opencv-python pillow spacy apscheduler
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 53.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [11]:
import os

# Path to dataset folder
dataset_path = '/content/drive/MyDrive/Dataset/New folder - winu/prescription data (images)'

# Load file names
files = [f for f in os.listdir(dataset_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
print(f"Loaded {len(files)} files from the dataset.")


Loaded 6 files from the dataset.


In [13]:
from PIL import Image
import pytesseract
!sudo apt install tesseract-ocr

def extract_text(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text


for file in files:
    file_path = os.path.join(dataset_path, file)
    text = extract_text(file_path)
    print(f"Extracted text from {file}:\n{text}\n")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (3,275 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [14]:
import spacy

# Load NLP model
nlp = spacy.load('en_core_web_sm')

def parse_prescription(text):
    doc = nlp(text)
    drugs = []
    for ent in doc.ents:
        if ent.label_ in ("DRUG", "QUANTITY", "TIME"):
            drugs.append((ent.text, ent.label_))
    return drugs


sample_text = "Take Paracetamol 500mg every 6 hours after food."
parsed_info = parse_prescription(sample_text)
print("Parsed prescription details:", parsed_info)


Parsed prescription details: [('6 hours', 'TIME')]


In [15]:
from datetime import datetime, timedelta

def generate_schedule(parsed_info):
    schedule = []
    now = datetime.now()

    for drug, label in parsed_info:
        if label == "TIME":
            interval = int(drug.split()[1])  # Example: 'every 6 hours'
            for i in range(4):  # Generate schedule for 4 intervals
                schedule.append(now + timedelta(hours=interval * i))
    return schedule

# Example usage
schedule = generate_schedule([("every 6 hours", "TIME")])
print("Drug schedule:", schedule)


Drug schedule: [datetime.datetime(2024, 12, 3, 6, 42, 31, 998601), datetime.datetime(2024, 12, 3, 12, 42, 31, 998601), datetime.datetime(2024, 12, 3, 18, 42, 31, 998601), datetime.datetime(2024, 12, 4, 0, 42, 31, 998601)]


In [16]:
from apscheduler.schedulers.background import BackgroundScheduler

scheduler = BackgroundScheduler()

def reminder_task(drug_name):
    print(f"Reminder: Time to take {drug_name}!")

# Add jobs for each drug in the schedule
for time in schedule:
    scheduler.add_job(reminder_task, 'date', run_date=time, args=["Paracetamol"])

scheduler.start()


In [ ]:
import time

adherence_log = {}

def log_adherence(drug_name, status):
    adherence_log[drug_name] = status
    if status == "missed":
        print("Re-triggering reminder...")
        scheduler.add_job(reminder_task, 'interval', minutes=5, args=[drug_name])


log_adherence("Paracetamol", "missed")
time.sleep(300)  # Simulate 5 minutes
log_adherence("Paracetamol", "consumed")


Re-triggering reminder...
Reminder: Time to take Paracetamol!
Reminder: Time to take Paracetamol!


In [18]:
import pandas as pd

# Save adherence logs to CSV
adherence_df = pd.DataFrame(list(adherence_log.items()), columns=["Drug", "Status"])
adherence_df.to_csv('adherence_log.csv', index=False)
print("Adherence log saved.")


Adherence log saved.
